In [ ]:
import json
import numpy as np
from pathlib import Path

meta = json.loads(Path("/content/meta.json").read_text(encoding="utf-8"))
print("Meta:", meta)

chunks = json.loads(Path("/content/chunks.json").read_text(encoding="utf-8"))
emb = np.load("/content/embeddings.npy")

print("n_chunks (meta):", meta.get("n_chunks"))
print("n_chunks (json):", len(chunks))
print("embeddings shape:", emb.shape)

if emb.shape[0] != len(chunks):
    print("WARNING: number of embeddings != number of chunks!")
else:
    print("OK: counts match.")

expected_dim = meta.get("embedding_dim")
if expected_dim and emb.shape[1] != expected_dim:
    print(f"WARNING: embedding dim mismatch (meta {expected_dim} != actual {emb.shape[1]})")
else:
    print("OK: embedding dimension matches meta.")

print("\n--- sample chunk (first, 400 chars) ---\n")
print(chunks[0]["text"][:400])

Meta: {'chunks_path': 'chunks.json', 'embeddings_path': 'embeddings.npy', 'n_chunks': 62, 'embedding_dim': 384, 'source_text_path': '/content/texto.txt', 'model_name': 'paraphrase-multilingual-MiniLM-L12-v2'}
n_chunks (meta): 62
n_chunks (json): 62
embeddings shape: (62, 384)
OK: counts match.
OK: embedding dimension matches meta.

--- sample chunk (first, 400 chars) ---

RESOLUÇÃO Nº 1.035-COPP/UFMS, DE 23 DE JUNHO DE 2025.
Aprova o Regulamento dos Cursos de Pós-Graduação
Stricto Sensu da Fundação Universidade Federal de
Mato Grosso do Sul.
 
O CONSELHO DE PESQUISA E PÓS-GRADUAÇÃO da Fundação Universidade
Federal de Mato Grosso do Sul, no uso da atribuição que lhe confere o art. 5º, caput, inciso IX,
do Regimento Geral da UFMS, aprovado pela Resolução nº 137, Coun


In [ ]:
!pip install chromadb[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.8 MB/s eta 

In [ ]:
import chromadb

In [ ]:
import json
import numpy as np
import chromadb
from pathlib import Path

chunks = json.loads(Path("/content/chunks.json").read_text(encoding="utf-8"))
emb = np.load("/content/embeddings.npy")

ids = [c["id"] for c in chunks]
documents = [c["text"] for c in chunks]
metadatas = [{"source": "regulamento_ufms", "index": i} for i in range(len(chunks))]

client = chromadb.PersistentClient(path="/content/chroma_db")

collection = client.get_or_create_collection(
    name="facom_regulamento",
    metadata={"source": "ufms_regulamento"}
)

collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas,
    embeddings=emb.tolist()
)

print(f"Indexed {len(ids)} chunks into Chroma collection 'facom_regulamento'")
print(f"Collection count: {collection.count()}")

Indexed 62 chunks into Chroma collection 'facom_regulamento'
Collection count: 62


In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
import numpy as np

embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device="cpu")

client = chromadb.PersistentClient(path="/content/chroma_db")

collection = client.get_collection("facom_regulamento")

def embed_query(q):
    vec = embed_model.encode([q], convert_to_numpy=True)
    return vec[0].tolist()

q = "Quando entra em vigor essa resolução?"
q_vec = embed_query(q)

res = collection.query(
    query_embeddings=[q_vec],
    n_results=3,
    include=["documents", "metadatas", "distances"]
)

print("=== RESULTADOS DA BUSCA ===\n")
for i, (doc, meta, dist) in enumerate(zip(res['documents'][0], res['metadatas'][0], res['distances'][0])):
    print(f"Resultado {i+1} (distância: {dist:.4f}):")
    print(f"Metadata: {meta}")
    print(f"Documento: {doc[:200]}...")
    print("-" * 80)
    print()

=== RESULTADOS DA BUSCA ===

Resultado 1 (distância: 12.0564):
Metadata: {'source': 'regulamento_ufms', 'index': 1}
Documento: Art. 3º Esta Resolução entra em vigor em 1º de janeiro de 2026.
 
 
FABRICIO DE OLIVEIRA FRAZILIO,
Presidente.
 
ANEXO – REGULAMENTO DOS CURSOS DE PÓS-GRADUAÇÃO STRICTO SENSU DA UFMS
(Resolução nº 1.0...
--------------------------------------------------------------------------------

Resultado 2 (distância: 14.3604):
Metadata: {'index': 50, 'source': 'regulamento_ufms'}
Documento: ocorrer em sessão fechada.
 
Art. 71. A avaliação será realizada pelos membros da Banca, com parecer, e
resultado:
I - aprovação; ou
II - reprovação.
10/07/2025, 16:20
SEI/UFMS - 5701360 - Resolução
h...
--------------------------------------------------------------------------------

Resultado 3 (distância: 14.5121):
Metadata: {'source': 'regulamento_ufms', 'index': 53}
Documento: da Banca e resultar em uma das seguintes decisões:
I - aprovação;
II - aprovação com revisão; ou
III - 